# Run MCube on the DLPFC dataset

In [1]:
setwd("/import/home/share/zw/pql")
getwd()

set.seed(20240709)

library(RhpcBLASctl)
max_cores <- 36
blas_set_num_threads(max_cores)

library(doParallel)
library(foreach)

library(Matrix)

library(ggplot2)
library(tiff)
library(grid)

[1] "/import/home/share/zw/pql"

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

Warning message:
“package ‘ggplot2’ was built under R version 4.3.3”


In [2]:
source("/import/home/share/zw/pql/code/mcube/mcubeClass.R")
source("/import/home/share/zw/pql/code/mcube/mcubeFitNull.R")
source("/import/home/share/zw/pql/code/mcube/mcubeScoreTest.R")
source("/import/home/share/zw/pql/code/mcube/mcubeUtils.R")
source("/import/home/share/zw/pql/code/mcube/mcubeKernel.R")
source("/import/home/share/zw/pql/code/mcube/mcubePlot.R")
source("/import/home/share/zw/pql/code/mcube/mcubeHelpers.R")

In [3]:
RAW_DATA_PATH <- "/import/home/share/zw/data/DLPFC"
DATA_PATH <- "/import/home/share/zw/pql/data/DLPFC"
RESULT_PATH <- "/import/home/share/zw/pql/results/DLPFC"

if (!dir.exists(file.path(RESULT_PATH))) {
    dir.create(file.path(RESULT_PATH), recursive = TRUE)
}

In [ ]:
n_slices <- 4
slice_idx_seq <- c(151673, 151674, 151675, 151676)

reference_ST <- data.matrix(
    read.csv(file.path(DATA_PATH, "reference_ST.csv"),
        header = TRUE, row.names = 1, check.names = FALSE
    )
)
dim(reference_ST)
num_celltypes <- nrow(reference_ST)

proportion_ST <- matrix(, nrow = 0, ncol = num_celltypes)
sample_id <- vector("character")
for(i in 1:n_slices){
    proportion_i <- as.matrix(read.csv(
        file.path(
            DATA_PATH,
            paste0("prop_slice", i - 1, ".csv")
        ),
        header = TRUE, row.names = 1, check.names = FALSE
    ))
    proportion_ST <- rbind(proportion_ST, proportion_i)
    sample_id <- c(sample_id, rep(paste0("slice_", slice_idx_seq[i]), nrow(proportion_i)))
}
dim(proportion_ST)
num_spots <- nrow(proportion_ST)
spots <- rownames(proportion_ST)
names(sample_id) <- spots


counts <- as.data.frame(readr::read_csv(
    file.path(DATA_PATH, "counts.csv")
))
rownames(counts) <- counts[, 1]
counts[, 1] <- NULL
counts <- counts[spots, ]
counts <- data.matrix(counts)
dim(counts)

coordinates <- as.matrix(read.csv(
    file.path(DATA_PATH, "3D_coordinates.csv"),
    header = TRUE, row.names = 1, check.names = FALSE
))
coordinates <- coordinates[spots, ]
dim(coordinates)

spot_effects_ST <- data.matrix(read.csv(
    file.path(DATA_PATH, "spot_effects_ST.csv"),
    header = TRUE, row.names = 1, check.names = FALSE
))[spots, 1]

platform_effects_ST <- data.matrix(read.csv(
    file.path(DATA_PATH, "platform_effects_ST.csv"),
    header = TRUE, row.names = 1, check.names = FALSE
))
rownames(platform_effects_ST) <- paste0("slice_", slice_idx_seq)

library_size_ST <- data.matrix(read.csv(
    file.path(DATA_PATH, "library_size_ST.csv"),
    header = TRUE, row.names = 1, check.names = FALSE
))[spots, 1]

[1]   18 4558

[1] 14243    18

New names:
• `` -> `...1`


# Applying MMM to each ST slice seperately

In [ ]:
for (slice_idx in slice_idx_seq) {
    spots_used <- which(sample_id == paste0("slice_", slice_idx))
    mcube_object <- createMCUBE(
        counts = counts[spots_used, ],
        coordinates = coordinates[spots_used, c("x", "y")],
        proportion = proportion_ST[spots_used, ],
        library_size = library_size_ST[spots_used],
        covariates = NULL, sample_id = sample_id[spots_used],
        reference = reference_ST,
        spot_effects = spot_effects_ST[spots_used],
        platform_effects = NULL, # Estimate platform effects automatically in the single slice case
        reference_threshold = 0.5,
        project = paste0("DLPFC_", slice_idx),
    )
    mcube_object <- mcubeFitNull(mcube_object, max_cores = max_cores)
    mcube_object <- mcubeTest(mcube_object, max_cores = max_cores)

    p <- mcubePlotPvalues(mcube_object@pvalues, "combined_pvalue", nrow = 2)
    p <- p +
        labs(title = NULL) +
        theme(
            text = element_text(size = 16),
            plot.title = element_text(size = 20),
            axis.title = element_text(size = 20),
            axis.text.x = element_text(size = 16),
            axis.text.y = element_text(size = 16)
        )
    ggsave(
        filename = file.path(
            RESULT_PATH,
            paste0("pvalue_qqplot_", slice_idx, ".png")
        ),
        plot = p, width = 12, height = 7
    )

    saveRDS(
        mcube_object,
        file = file.path(
            RESULT_PATH,
            paste0("mcube_", slice_idx, ".rds")
        )
    )
    # mcube_object <- readRDS(
    #     file = file.path(
    #         RESULT_PATH,
    #         paste0("mcube_", slice_idx, ".rds")
    #     )
    # )

    assign(
        paste0("mcube_object_", slice_idx),
        mcube_object
    )
    rm(mcube_object)
}

In [141]:
for (slice_idx in slice_idx_seq) {
    # spots_used <- which(sample_id == paste0("slice", slice_idx))
    # mcube_object <- createMCUBE(
    #     counts = counts[spots_used, ],
    #     coordinates = coordinates[spots_used, c("x", "y")],
    #     proportion = proportion_ST[spots_used, ],
    #     library_size = library_size_ST[spots_used],
    #     covariates = NULL, sample_id = sample_id[spots_used],
    #     reference = reference_ST,
    #     spot_effects = spot_effects_ST[spots_used],
    #     platform_effects = NULL,
    #     reference_threshold = 0.3,
    #     project = paste0("DLPFC_", slice_idx),
    # )
    # mcube_object <- mcubeFitNull(mcube_object, max_cores = max_cores)
    # mcube_object <- mcubeTest(mcube_object, max_cores = max_cores)

    # # p <- mcubePlotPvalues(mcube_object@pvalues, "combined_pvalue", nrow = 2)
    # # p <- p +
    # #     labs(title = NULL) +
    # #     theme(
    # #         text = element_text(size = 16),
    # #         plot.title = element_text(size = 20),
    # #         axis.title = element_text(size = 20),
    # #         axis.text.x = element_text(size = 16),
    # #         axis.text.y = element_text(size = 16)
    # #     )
    # # ggsave(
    # #     filename = file.path(
    # #         RESULT_PATH,
    # #         paste0("pvalue_qqplot_", slice_idx, ".png")
    # #     ),
    # #     plot = p, width = 12, height = 7
    # # )

    # saveRDS(
    #     mcube_object,
    #     file = file.path(
    #         RESULT_PATH,
    #         paste0("mcube_", slice_idx, ".rds")
    #     )
    # )
    mcube_object <- readRDS(
        file = file.path(
            RESULT_PATH,
            paste0("mcube_", slice_idx, ".rds")
        )
    )

    assign(
        paste0("mcube_object_", slice_idx),
        mcube_object
    )
    rm(mcube_object)
}

## Visualization of the deconvolution results of slice 1

In [15]:
slice_idx <- slice_idx_seq[1]
mcube_object <- get(paste0("mcube_object_", slice_idx))

In [ ]:
# Proportion
proportion_df <- reshape2::melt(
    mcube_object@proportion[, mcube_object@celltype_test],
    varnames = c("spot", "celltype"), value.name = "proportion"
)

# Boxplot
p <- ggplot(proportion_df, aes(x = celltype, y = proportion)) +
  geom_boxplot(aes(fill = celltype), alpha = 0.5, show.legend = FALSE) +
  theme_classic() +
    labs(title = NULL, x = NULL, y = "Proportion") +
    theme(
      text = element_text(size = 16),
      axis.text.x = element_text(size = 12, angle = 45, hjust = 1)
    )
ggsave(
  filename = file.path(RESULT_PATH, "proportion.pdf"),
  plot = p, width = 4, height = 4
)
p

In [ ]:
# spot effects
spot_effects_df <- data.frame(
    x = mcube_object@coordinates[, 1],
    y = mcube_object@coordinates[, 2],
    spot_effect = mcube_object@spot_effects
)

p <- ggplot(spot_effects_df, aes(x = x, y = y, color = spot_effect)) +
  geom_point(size = 1) +
  scale_color_gradient2(
    low = "#64AE11", mid = "white", high = "#FD3000",
    midpoint = median(spot_effects_df$spot_effect)
  ) +
  scale_y_reverse() +
    coord_fixed(ratio = 1) +
    labs(title = NULL, x = NULL, y = NULL) +
    theme_classic() +
    theme(
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      legend.position = "None"
    )
ggsave(
  filename = file.path(RESULT_PATH, "spot_effects.pdf"),
  plot = p, width = 4, height = 4
)
p

In [ ]:
# Platform effects
p <- ggplot(data = NULL, aes(x = platform_effects_ST[1, mcube_object@gene_test])) +
  geom_histogram(bins = 20, fill = "lightblue", color = "black") +
  xlab("Platform effect") +
  ylab("Count") +
    theme_classic() +
    theme(
      text = element_text(size = 16),
      axis.text = element_text(size = 12)
    )
ggsave(
  filename = file.path(RESULT_PATH, "platform_effects.pdf"),
  plot = p, width = 4, height = 4
)
p
# ggsave(file.path(paste0(PICTURE_PATH,'platform_effects.eps')), width = 5, height = 5, device = cairo_ps)

In [12]:
rm(mcube_object)

## Visualization of the cell type-specific SVG results

In [ ]:
celltype <- "Ex_5_L5"
demo_genes <- c("COX6C", "CAMK2N1", "ENC1", "SYT1")
for(i in 1:n_slices){
    mcube_object <- get(paste0("mcube_object_", slice_idx_seq[i]))

    # Since STitch3D aligns multiple ST slices, the integrated 3D coordinates cannot match the H&E image
    # Therefore, we use the original 2D coordinates instead
    coordinates_original <- as.matrix(read.csv(
        file.path(DATA_PATH, paste0("2D_coordinates_slice", i - 1, ".csv")),
        header = TRUE, row.names = 1, check.names = FALSE
    ))
    rownames(coordinates_original) <- paste0(rownames(coordinates_original), "-slice", i - 1)
    coordinates_original <- coordinates_original[rownames(mcube_object@proportion), ]

    # Load in H&E image
    he_image <- tiff::readTIFF(
        file.path(
            RAW_DATA_PATH, "spatialLIBD", slice_idx_seq[i],
            paste0(slice_idx_seq[i], "_full_image.tif")
        )
    )
    he_image_0.5 <- array(0.5, dim = c(dim(he_image)[1], dim(he_image)[2], 4))
    he_image_0.5[, , 1] <- he_image[, , 1]
    he_image_0.5[, , 2] <- he_image[, , 2]
    he_image_0.5[, , 3] <- he_image[, , 3]
    # he_image_0.5[, , 4] <- 0.5 

    boader <- 500
    xlim <- c(
        min(coordinates_original[, 1]) - boader, 
        max(coordinates_original[, 1]) + boader
    )
    ylim <- c(
        min(coordinates_original[, 2]) - boader,
        max(coordinates_original[, 2]) + boader
    )

    # Proportion of cell type
    p <- mcubePlotPropCellType(
        mcube_object@proportion, coordinates_original, celltype,
        he_image = he_image_0.5[ylim[1]:ylim[2], xlim[1]:xlim[2], ], background = FALSE,
        xlim = xlim, ylim = ylim
    ) +
      scale_x_continuous(limits = xlim, expand = c(0, 0)) +
      scale_y_continuous(trans = "reverse", limits = rev(ylim), expand = c(0, 0)) +
      theme_void() +
      theme(
          plot.title = element_blank(),
          axis.title = element_blank(),
          axis.text = element_blank(),
          axis.ticks = element_blank(),
          panel.grid = element_blank(),
          legend.position = "none"
      )
    ggsave(
        filename = file.path(
            RESULT_PATH,
            paste0("proportion_", celltype, "_slice", slice_idx_seq[i], ".png")
        ),
        plot = p, width = 4, height = 4, bg = "white"
    )
    
    # Spatial pattern of SVGs
    for (gene in demo_genes) {
        mcube_object@coordinates <- coordinates_original
        p <- mcubePlotExprCellTypeBinary(
            mcube_object,
            celltype = celltype, gene = gene,
            he_image = he_image_0.5[ylim[1]:ylim[2], xlim[1]:xlim[2], ], background = FALSE,
            xlim = xlim, ylim = ylim
        ) +
            scale_x_continuous(limits = xlim, expand = c(0, 0)) +
            scale_y_continuous(trans = "reverse", limits = rev(ylim), expand = c(0, 0)) +
            theme_void() +
            theme(
                plot.title = element_blank(),
                axis.title = element_blank(),
                axis.text = element_blank(),
                axis.ticks = element_blank(),
                panel.grid = element_blank(),
                legend.position = "none"
            )
        ggsave(
            filename = file.path(
                RESULT_PATH,
                paste0(celltype, "_", gene, "_slice", slice_idx_seq[i], ".png")
            ),
            plot = p, width = 4, height = 4, bg = "white"
        )
    }

    rm(mcube_object)
}

## Negative control using slice 1

In [ ]:
mcube_object <- get(paste0("mcube_object_", slice_idx_seq[1]))

shuffle_idx <- sample(
    1:nrow(mcube_object@coordinates),
    nrow(mcube_object@coordinates)
)
write.csv(shuffle_idx, file = file.path(DATA_PATH, "shuffle_idx.csv"), row.names = FALSE)
shuffle_idx <- read.csv(file.path(DATA_PATH, "shuffle_idx.csv"), header = TRUE)[, 1]
length(unique(shuffle_idx))

coordinates_shuffle <- mcube_object@coordinates[shuffle_idx, ]
rownames(coordinates_shuffle) <- rownames(mcube_object@coordinates)
write.csv(coordinates_shuffle, file = file.path(DATA_PATH, "coordinates_shuffle.csv"))
coordinates_shuffle <- data.matrix(read.csv(
    file.path(DATA_PATH, "coordinates_shuffle.csv"),
    header = TRUE, row.names = 1, check.names = FALSE
))

In [ ]:
mcube_object_null <- mcube_object
rm(mcube_object)
mcube_object_null@coordinates <- coordinates_shuffle
mcube_object_null <- mcubeFitNull(mcube_object_null, max_cores = max_cores)
mcube_object_null <- mcubeTest(mcube_object_null, max_cores = max_cores)

p <- mcubePlotPvalues(mcube_object_null@pvalues, "combined_pvalue", nrow = 2)
ggsave(
    filename = file.path(
        RESULT_PATH,
        "pvalue_qqplot_1_null.png"
    ),
    plot = p, width = 12, height = 7
)

pvalues_null <- mcube_object_null@pvalues
rm(mcube_object_null)
saveRDS(
    pvalues_null,
    file = file.path(
        RESULT_PATH,
        paste0("pvalues_", slice_idx_seq[1], "_null.rds")
    )
)
# pvalues_null <- readRDS(
#     file = file.path(
#         RESULT_PATH,
#         "pvalues_1_null.rds"
#     )
# )

In [23]:
rm(list = ls(pattern = "mcube_object_"))

# Applying MMM to the integrated four ST slices together

In [ ]:
mcube_object_all <- createMCUBE(
    counts = counts, coordinates = coordinates, 
    proportion = proportion_ST, library_size = library_size_ST, 
    covariates = NULL, sample_id = sample_id,
    reference = reference_ST,
    spot_effects = spot_effects_ST, platform_effects = platform_effects_ST,
    project = "DLPFC_all"
)
mcube_object_all <- mcubeFitNull(mcube_object_all, max_cores = max_cores)
mcube_object_all <- mcubeTest(mcube_object_all, max_cores = max_cores)

p <- mcubePlotPvalues(mcube_object_all@pvalues, "combined_pvalue", nrow = 2) +
    labs(title = NULL) +
    theme(
        text = element_text(size = 16),
        plot.title = element_text(size = 20),
        axis.title = element_text(size = 20),
        axis.text.x = element_text(size = 16),
        axis.text.y = element_text(size = 16)
    )
ggsave(
    filename = file.path(
        RESULT_PATH,
        paste0("pvalue_qqplot_all.png")
    ),
    plot = p, width = 12, height = 7
)
p

saveRDS(
    mcube_object_all,
    file = file.path(
        RESULT_PATH,
        paste0("mcube_all", ".rds")
    )
)
# mcube_object_all <- readRDS(
#     file = file.path(
#         RESULT_PATH,
#         paste0("mcube_all", ".rds")
#     )
# )

In [5]:
mcube_object_all <- readRDS(
    file = file.path(
        RESULT_PATH,
        paste0("mcube_all", ".rds")
    )
)

In [6]:
# For visualization
mcube_object_all@coordinates[, "x"] = -mcube_object_all@coordinates[, "x"]
mcube_object_all@coordinates[, "z"] = (mcube_object_all@coordinates[, "z"] -
    (mcube_object_all@coordinates[, "z"] > 14) * 397.3) * 200
mcube_object_all@coordinates[, "x"] = mcube_object_all@coordinates[, "x"] / 1.37
mcube_object_all@coordinates[, "y"] = mcube_object_all@coordinates[, "y"] / 1.37
mcube_object_all@coordinates[, "z"] = mcube_object_all@coordinates[, "z"] / 1.37

In [7]:
# demo_genes <- c("COX6C", "TMSB10")
celltype <- "Ex_5_L5"
demo_genes <- c("COX6C", "CAMK2N1", "ENC1", "SYT1")
for (gene in demo_genes) {
    p <- mcubePlotExprCellType3D(
        mcube_object_all, celltype, gene,
        spot_size = 1.5, opacity_target = 0.7, opacity_background = 0.1,
        plotly_eye = list(x = -0.9, y = 1.7, z = 1.2)
    )
    plotly::save_image(
        p, file.path(RESULT_PATH, paste0(celltype, "_", gene, "_3D", ".pdf")),
        width = 500, height = 500, scale = 5
    )
}

In [ ]:
Reduce(
    intersect,
    list(
        rownames(mcube_object_1@pvalues[[celltype]])[order(mcube_object_1@pvalues[[celltype]][, 6])][1:10],
        rownames(mcube_object_2@pvalues[[celltype]])[order(mcube_object_2@pvalues[[celltype]][, 6])][1:10],
        rownames(mcube_object_3@pvalues[[celltype]])[order(mcube_object_3@pvalues[[celltype]][, 6])][1:10],
        rownames(mcube_object_4@pvalues[[celltype]])[order(mcube_object_4@pvalues[[celltype]][, 6])][1:10]
    )
)

In [ ]:
mcubePlotExprCellType3D(
        mcube_object_all, celltype, "COX6C",
        spot_size = 1.5, opacity_target = 0.7, opacity_background = 0.1,
        plotly_eye = list(x = -0.9, y = 1.7, z = 1.2)
    )